In [ ]:
import numpy as np
from maxvolpy.maxvol import maxvol
from numpy.polynomial import Chebyshev as T
import numpy.linalg as la
import matplotlib.pyplot as plt
from scipy.linalg import lu as lu
from gen_mat import *
from ids import *
from block_maxvol import *
from block_rect_maxvol import *
import scipy.linalg
%matplotlib inline

### variants to generate matrix $A$

In [ ]:
n = 5000 #nmber of sampling points   
deg = 9 #Degree of polynomial       
interval = [0, 5]  # interval of x
x= np.linspace(interval[0], interval[1], n, dtype=np.float64)
p_size = deg+1
nder = 1

In [ ]:
nder = 1 # dimension
n = 40 # number of points
n_test = 5000 #points on test grid
p_size = (nder+1)*5 #number of monoms (number of columns)
interval = [-2, 2]  # interval of x_test
x = np.random.rand(n, nder)


In [ ]:
c = GenMat(p_size, x,poly=cheb, poly_diff=cheb_diff, debug=False,pow_p=1)
print c.shape
# turn matrix to the block structure
A = matrix_prep(c,nder+1,n)
print A.shape

In [ ]:
max_val = 15
n, p_size = 360, 12
mat_int = np.random.uniform( low=-max_val, high=max_val, size=n*p_size).reshape((n,p_size)).astype(int)
A = mat_int.astype(float)
nder = 2

In [ ]:
A = np.array([[-8., -7.,  1.,  0. ,-5.,  2.],
 [ 6., -1., -8.,  4.,  4., -9.],
 [ 9.,  5.,  8.,  0., -6., -4.],
 [ 6.,  5., -6.,  2., -9.,  8.],
 [ 3., -8.,  3., -3.,  4., -4.],
 [-7. , 2.,  0., -6.,  5., -5.],
 [ 6.  ,1., -1.,  0., -2.,  1.],
 [ 1.  ,9., -6., -4.,  0.,  0.],
 [-2.  ,0.,  0., -7., -6.,  4.],
 [-1.  ,0., -1.,  9., -4.,  4.],
 [ 4., -4. , 1.,  7., -6., -9.],
 [-7.,  8.,  4.,  1.,  0.,  0.]])
print A
nder = 1
p_size = A.shape[1]

### start information about loaded matrix

In [ ]:
print la.det(A[:A.shape[1]])
print la.matrix_rank(A)
_,s,_ = scipy.linalg.svd(A)
print s

piv,_ = maxvol(A)
print piv
print la.det(A[piv])

###  test PLUQ Preprocessing separately

In [ ]:
p,l,u,q,inf = pluq_ids(A,nder, debug=False)

In [ ]:
print(la.det(np.dot(perm_matrix(p),b)[:b.shape[1]]))

print p[:b.shape[1]]
print q

In [ ]:
#turn rows in every block in correct order, i.e. 2,1,3 -> 1,2,3; 
pr = p_preproc(p,nder+1)
print pr[:b.shape[1]]

### test Block maxvol perfomance separately

In [ ]:
%time r,t,y = block_maxvol(A_init,nder, tol=0.0,max_iters=200,swm_upd=True)

In [ ]:
%time C, B, P = block_maxvol(A_init, nder,tol=0.0,max_iters = 2000,swm_upd=False)

In [ ]:
print np.linalg.det((b[:b.shape[1]]))
print np.linalg.det((A_init[:b.shape[1]]))
print np.linalg.det((r[:b.shape[1]]))
print np.linalg.det((C[:b.shape[1]]))

In [ ]:
print y[:b.shape[1]] # swm permuts
print P[:b.shape[1]] # recalc A^{-1} permuts
print pr[:b.shape[1]]

### test rect_block_maxvol

In [ ]:
piv = rect_block_maxvol(A, nder, Kmax = 21, max_iters=100,rect_tol = 0.05, tol = 0.0, debug = False, ext_debug = False)